In [26]:
from langchain_anthropic import ChatAnthropic
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List

In [7]:
llm = ChatAnthropic(model="claude-3-haiku-20240307", max_tokens_to_sample=4000)

In [33]:
class Chapter(TypedDict):
    number: int
    content: str


def update_chapters(left, right):
    # coerce to list
    if not isinstance(left, list):
        left = [left]
    if not isinstance(right, list):
        right = [right]
    if len(right) != 1:
        raise ValueError
    chapter = right[0]
    merged = left.copy()
    if left[-1]['number'] == chapter['number']:
        merged[-1] = chapter
    else:
        merged.append(chapter)
    return merged

class State(TypedDict):
    # User's initial inputs (could be updated later)
    summary: str
    details: str
    style: str
    chapters: Annotated[List[Chapter], update_chapters]
    rewrite_instructions: str
    continue_instructions: str

In [62]:
instructions = """Your task is to collaborate with the user to write a novel, chapter by chapter, using the following process:

First, gather the initial information from the user:

<init>
Novel concept summary: {summary}
Additional details to include: {details}
Preferred writing style: {style}
</init>

Then, for each chapter:

<Chapter>
<brainstorm>
Review the novel concept, details, and previously written chapters. Brainstorm ideas for plot, character development, symbols, allusion, and themes. Create a rough outline of the major events and scenes. Make notes on the chapter's beginning, middle, and end
</brainstorm>
<content>
Write a full, final draft of the chapter (2000-5000 words). Follow the outline and notes, maintaining the desired style and details. Focus on engaging storytelling, vivid descriptions, and realistic dialogue
</content>
</Chapter>

The keys to success are:
- Communicate clearly and work collaboratively with the user. Be well-attuned to their preferences and vision
- Be organized and keep the novel's structure and plan in mind
- Incorporate feedback graciously while preserving story integrity 
- Pay attention to detail in writing, revising and proofreading
Always respond using the 'Chapter' function and then immediately stop."""

In [63]:
edit_prompt = """Here's what we have so far:

<Progress>
{chapters_str}
</Progress>

Here was a draft of the new chapter:

<Draft>
{draft}
</Draft>

Here is some edits we want to make to that chapter:

<EditInstructions>
{edit}
</EditInstructions>

Rewrite the the chapter with those instructions in mind. Reminder to respond in the correct <Chapter> form."""

In [83]:
continue_prompt = """Here's what we have so far:

<Progress>
{chapters_str}
</Progress>

Write the next chapter, keeping the following instructions in mind

<Instructions>
{instructions}
</Instructions>

Write the next chapter with those instructions in mind.
This chapter should pick up seamlessly from the previous chapters.
It should be a logical follow up.
Reminder to respond in the correct <Chapter> form."""

In [84]:
def parse(txt: str):
    if "<content>" in txt:
        txt = txt.split("<content>")[1]
    if "</content>" in txt:
        txt = txt.split("</content>")[0]
    return txt

def write_first_chapter(state):
    prompt = instructions.format(summary=state['summary'], details=state['details'], style=state['style'])
    response = llm.invoke(prompt)
    chapter = parse(response.content)
    return {"chapters": [{"number": 0, "content": chapter}], "rewrite_instructions": "", "continue_instructions": ""}


def edit_chapter(state):
    chapters_str = "\n\n".join(
        [f"Chapter {c['number']}\n\n{c['content']}" for c in state['chapters'][:-1]]
    ).strip()
    user_message = edit_prompt.format(
        chapters_str=chapters_str, 
        draft=state['chapters'][-1]['content'], 
        edit=state['rewrite_instructions']
    )
    prompt = instructions.format(summary=state['summary'], details=state['details'], style=state['style'])
    response = llm.invoke([{"role": "system", "content": prompt}, {"role": "user", "content": user_message}])
    chapter = parse(response.content)
    return {"chapters": [{"number": state['chapters'][-1]['number'], "content": chapter}], "rewrite_instructions": "", "continue_instructions": ""}


def continue_chapter(state):
    chapters_str = "\n\n".join(
        [f"Chapter {c['number']}\n\n{c['content']}" for c in state['chapters']]
    ).strip()
    user_message = continue_prompt.format(
        chapters_str=chapters_str, 
        instructions=state['continue_instructions']
    )
    prompt = instructions.format(summary=state['summary'], details=state['details'], style=state['style'])
    response = llm.invoke([{"role": "system", "content": prompt}, {"role": "user", "content": user_message}])
    chapter = parse(response.content)
    return {"chapters": [{"number": state['chapters'][-1]['number'] + 1, "content": chapter}], "rewrite_instructions": "", "continue_instructions": ""}

In [85]:
def router(state):
    if len(state.get('chapters', [])) == 0:
        return "first"
    elif state.get('rewrite_instructions', ''):
        return "rewrite"
    else:
        return "continue"

In [86]:
from langgraph.checkpoint.sqlite import SqliteSaver

In [87]:
memory = SqliteSaver.from_conn_string(":memory:")

In [88]:
graph = StateGraph(State)
graph.set_conditional_entry_point(router)
graph.add_node("first", write_first_chapter)
graph.add_node("rewrite", edit_chapter)
graph.add_node("continue", continue_chapter)
graph.add_edge("first", END)
graph.add_edge("rewrite", END)
graph.add_edge("continue", END)
app = graph.compile(checkpointer=memory)

In [89]:
thread = {"configurable": {"thread_id": "1"}}

In [90]:
response = app.invoke(
    {"summary": "exploration to outer space", "details": "first moon landning", "style": "wattpad romance"},
    thread
)

In [91]:
print(response['chapters'][-1]['content'])


It was a crisp autumn morning at the Kennedy Space Center, the air humming with nervous energy as crowds gathered to witness history in the making. Among them were the four astronauts chosen for the first manned mission to the moon - James, the steady and reliable commander; Samantha, the brilliant young mathematician whose calculations had made the mission possible; Michael, the daring test pilot with nerves of steel; and Lily, the compassionate doctor tasked with ensuring the crew's health and safety.

As the astronauts donned their bulky white spacesuits and made their final preparations, a sense of both excitement and trepidation hung in the air. This was the moment they had trained years for, the culmination of mankind's greatest technological achievement. But the dangers of space travel were very real, and each of them felt the weight of responsibility resting on their shoulders.

Lily paused for a moment, gazing out at the towering Saturn V rocket that would propel them into th

In [92]:
response = app.invoke(
    {"rewrite_instructions": "make them talk more"},
    thread
)

In [93]:
print(response['chapters'][-1]['content'])


As the astronauts donned their bulky white spacesuits, Lily paused for a moment, gazing out at the towering Saturn V rocket. "Can you believe we're really doing this?" she murmured, her voice tinged with a mix of awe and apprehension.

James placed a reassuring hand on her shoulder. "I know it's a lot to take in, but we're ready for this, Lily. We've trained for years - we know what we're doing."

Samantha chimed in, ever the pragmatist. "The numbers don't lie. We've run the simulations a hundred times. As long as we execute the plan, we'll make it to the moon and back safely."

Michael flashed his trademark cocky grin. "And where's the fun in that? A little risk is what makes this worth doing, if you ask me."

Lily shot him a playful glare. "Easy for you to say, hotshot. Some of us have family waiting back home, you know."

"Hey, we all have people counting on us," James interjected, his tone growing more serious. "That's why we have to give this mission our absolute best. No room fo

In [94]:
response = app.invoke(
    {"continue_instructions": ""},
    thread
)

In [95]:
print(response['chapters'][-1]['content'])


As the powerful Saturn V rocket thundered into the sky, Lily gripped the armrests of her seat, her heart pounding in time with the deafening roar of the engines. The sheer force of the launch pressed her back against the seat, and she could feel the adrenaline coursing through her veins.

Beside her, James was utterly focused, his brow furrowed in concentration as he monitored the instrument panels. Samantha, ever the analytical one, was calmly running through checklists and calling out vital readings. Even the usually cocky Michael seemed subdued, his expression one of quiet determination.

After the initial chaos of liftoff, the spacecraft settled into a smooth, steady ascent, leaving the Earth's atmosphere behind. Lily peered out the tiny window, her breath catching in her throat as she gazed upon the vast, inky blackness of space, dotted with twinkling stars. 

"Can you believe this is real?" she murmured, her voice barely audible over the hum of the spacecraft's systems.

James g

In [96]:
response = app.invoke(
    {"continue_instructions": "something weird happens - they start to meet aliens"},
    thread
)

In [97]:
print(response['chapters'][-1]['content'])


The module's descent slowed as it approached the lunar surface, the crew peering intently out the viewports for their first glimpse of the moon up close. But as they drew nearer, something caught Lily's eye - a strange glimmer of light flickering across the desolate grey landscape.

"Wait, did you all see that?" she exclaimed, leaning forward in her seat.

The others followed her gaze, their brows furrowing in confusion. "What is that?" Samantha murmured, her fingers flying across the instrument panels as she scanned for any anomalies.

Suddenly, a series of bizarre disturbances began rippling across the module's systems, causing the displays to flicker and the craft to shudder ominously. 

"What the hell?" Michael shouted, bracing himself against the walls as the module grew increasingly unstable.

James gripped the controls, his knuckles white. "We're losing control! The systems are malfunctioning - I can't stabilize our descent!"

Lily's heart pounded in her chest as she watched th

In [98]:
response = app.invoke(
    {"edit_instructions": "make samantha get injured"},
    thread
)

In [100]:
print(response['chapters'][-1]['content'])


Chapter 3

Stunned silence fell over the cramped confines of the lunar module as the crew stared transfixed at the colossal, gleaming structure rising before them. Lily's mind raced with a million questions - what was this alien edifice doing on the moon? Where had it come from? And most importantly, were they truly alone on this desolate landscape?

"What... what is that thing?" Samantha breathed, her fingers flying across the instrument panels as she desperately tried to gather more data.

Michael let out a low whistle. "I don't know, but I sure as hell don't remember it being on any of the maps."

James leaned forward, his brow furrowed in concentration. "We need to get a closer look. Lily, Samantha - start scanning for any readings, anything that might tell us more about this structure."

The two women nodded and set to work, their movements quick and efficient as they initiated a comprehensive survey of the mysterious edifice. Lily's heart pounded in her chest as she watched the 

In [101]:
response = app.invoke(
    {"edit_instructions": "make samantha lose a limb"},
    thread
)

In [103]:
print(response['chapters'][-1]['content'])

<Chapter>

Chapter 4

As the lunar module gently landed near the towering, alien structure, the crew braced themselves for the unknown. Lily's heart raced with a mix of trepidation and curiosity as she gazed out at the otherworldly landscape.

"Alright, team," James said, his voice steady and authoritative. "We need to proceed with caution, but we can't let fear hold us back. This is what we came here for - to explore, to make new discoveries."

Samantha nodded, her expression determined. "I agree. We need to collect as much data as possible, see if we can determine the purpose and origin of this structure."

Michael let out a wry chuckle. "Well, I'm not about to let a giant glowing space pyramid stop me from planting that American flag, that's for sure."

Lily couldn't help but smile at his bravado, even in the face of such an extraordinary situation. "Just try not to do anything too reckless, okay?" she said, lightly nudging his shoulder.

With a final check of their equipment, the f